<a href="https://colab.research.google.com/github/tanjinadnanabir/Machine-Learning-Zoomcamp-2025/blob/main/HW_3_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import Libs

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mutual_info_score
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LogisticRegression

## Load the Dataset

In [6]:
df = pd.read_csv('https://raw.githubusercontent.com/alexeygrigorev/datasets/master/course_lead_scoring.csv')

In [56]:
df.head()

,lead_source,industry,number_of_courses_viewed,annual_income,employment_status,location,interaction_count,lead_score,converted
0,paid_ads,NA,1,79450.0,unemployed,south_america,4,0.94,1
1,social_media,retail,1,46992.0,employed,south_america,1,0.80,0
2,events,healthcare,5,78796.0,unemployed,australia,3,0.69,1
3,paid_ads,retail,2,83843.0,NA,australia,1,0.87,0
4,referral,education,3,85012.0,self_employed,europe,3,0.62,1


## Data Preparation

In [8]:
df.columns

Index(['lead_source', 'industry', 'number_of_courses_viewed', 'annual_income',
       'employment_status', 'location', 'interaction_count', 'lead_score',
       'converted'],
      dtype='object')

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1462 entries, 0 to 1461
Data columns (total 9 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   lead_source               1334 non-null   object 
 1   industry                  1328 non-null   object 
 2   number_of_courses_viewed  1462 non-null   int64  
 3   annual_income             1281 non-null   float64
 4   employment_status         1362 non-null   object 
 5   location                  1399 non-null   object 
 6   interaction_count         1462 non-null   int64  
 7   lead_score                1462 non-null   float64
 8   converted                 1462 non-null   int64  
dtypes: float64(2), int64(3), object(4)
memory usage: 102.9+ KB


In [10]:
df.isna().sum()

,0
lead_source,128
industry,134
number_of_courses_viewed,0
annual_income,181
employment_status,100
location,63
interaction_count,0
lead_score,0
converted,0


In [11]:
df.dtypes

,0
lead_source,object
industry,object
number_of_courses_viewed,int64
annual_income,float64
employment_status,object
location,object
interaction_count,int64
lead_score,float64
converted,int64


In [12]:
df.columns[df.dtypes != 'object']

Index(['number_of_courses_viewed', 'annual_income', 'interaction_count',
       'lead_score', 'converted'],
      dtype='object')

In [13]:
numerical_cols = ['number_of_courses_viewed', 'annual_income', 'interaction_count','lead_score']
numerical_cols

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [14]:
df.columns[df.dtypes == 'object']

Index(['lead_source', 'industry', 'employment_status', 'location'], dtype='object')

In [15]:
categorical_cols = ['lead_source', 'industry', 'employment_status', 'location']

In [16]:
for c in categorical_cols:
    print(c, df[c].nunique())

lead_source 5
industry 7
employment_status 4
location 7


In [17]:
for n in numerical_cols:
    df[n] = df[n].fillna(0)
for c in categorical_cols:
    df[c] = df[c].fillna('NA')

In [18]:
df.isna().sum()

,0
lead_source,0
industry,0
number_of_courses_viewed,0
annual_income,0
employment_status,0
location,0
interaction_count,0
lead_score,0
converted,0


## Question 1

What is the most frequent observation (mode) for the column industry?

In [19]:
df['industry'].mode()

,industry
0,retail


In [20]:
df['industry'].isna().sum()

np.int64(0)

In [21]:
df['industry'].value_counts()

,count
industry,
retail,203
finance,200
other,198
healthcare,187
education,187
technology,179
manufacturing,174
NA,134


retail

## Question 2

Create the correlation matrix for the numerical features of your dataset. In a correlation matrix, you compute the correlation coefficient between every pair of features.

What are the two features that have the biggest correlation?

In [22]:
df[numerical_cols].corr()

,number_of_courses_viewed,annual_income,interaction_count,lead_score
number_of_courses_viewed,1.000000,0.009770,-0.023565,-0.004879
annual_income,0.009770,1.000000,0.027036,0.015610
interaction_count,-0.023565,0.027036,1.000000,0.009888
lead_score,-0.004879,0.015610,0.009888,1.000000


In [23]:
np.array(list(filter(lambda x: x != 1, list(df[numerical_cols].corr().values.flatten())))).max()

np.float64(0.02703647240481443)

0.02703647240481443  
annual_income and interaction_count

## Split the data

- Split your data in train/val/test sets with 60%/20%/20% distribution.
- Use Scikit-Learn for that (the train_test_split function) and set the seed to 42.
- Make sure that the target value y is not in your dataframe.

In [25]:
df_full_train, df_test = train_test_split(df, test_size=0.2, random_state=42)

In [26]:
df_train, df_val = train_test_split(df_full_train, test_size=0.25, random_state=42)

In [27]:
len(df_train), len(df_val), len(df_test)

(876, 293, 293)

In [29]:
df_train = df_train.reset_index(drop=True)
df_val = df_val.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)

In [30]:
y_full_train = df_full_train.converted
y_train = df_train.converted
y_val = df_val.converted
y_test = df_test.converted

In [31]:
del df_full_train['converted']
del df_train['converted']
del df_val['converted']
del df_test['converted']

## Question 3

Calculate the mutual information score between y and other categorical variables in the dataset. Use the training set only.
- Round the scores to 2 decimals using round(score, 2).
- Which of these variables has the biggest mutual information score? [industry, location, lead_source,
employment_status]

In [32]:
for i in ['industry', 'location', 'lead_source', 'employment_status']:
    print(i, round(mutual_info_score(df_train[i], y_train), 2), mutual_info_score(df_train[i], y_train))

industry 0.01 0.011574521435657112
location 0.0 0.004464157884038034
lead_source 0.04 0.03539624379726594
employment_status 0.01 0.012937677269442782


lead_source 0.04

## Question 4

- Now let's train a logistic regression.
- Remember that we have several categorical variables in the dataset. Include them using one-hot encoding.
- Fit the model on the training dataset.
    - To make sure the results are reproducible across different versions of Scikit-Learn, fit the model with these parameters:
    - model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
- Calculate the accuracy on the validation dataset and round it to 2 decimal digits.

In [33]:
categorical_cols

['lead_source', 'industry', 'employment_status', 'location']

In [34]:
numerical_cols

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [35]:
dicts_train = df_train[categorical_cols+numerical_cols].to_dict(orient='records')

In [36]:
dv = DictVectorizer(sparse=False)
dv.fit(dicts_train)

DictVectorizer(sparse=False)

In [37]:
dv.get_feature_names_out()

array(['annual_income', 'employment_status=NA',
       'employment_status=employed', 'employment_status=self_employed',
       'employment_status=student', 'employment_status=unemployed',
       'industry=NA', 'industry=education', 'industry=finance',
       'industry=healthcare', 'industry=manufacturing', 'industry=other',
       'industry=retail', 'industry=technology', 'interaction_count',
       'lead_score', 'lead_source=NA', 'lead_source=events',
       'lead_source=organic_search', 'lead_source=paid_ads',
       'lead_source=referral', 'lead_source=social_media', 'location=NA',
       'location=africa', 'location=asia', 'location=australia',
       'location=europe', 'location=middle_east',
       'location=north_america', 'location=south_america',
       'number_of_courses_viewed'], dtype=object)

In [38]:
len(dv.get_feature_names_out())

31

In [39]:
s = 0
for c in categorical_cols:
    s+=df[c].nunique()
s+=len(numerical_cols)
print(s)

31


In [40]:
X_train = dv.transform(dicts_train)

In [41]:
dicts_val = df_val[categorical_cols+numerical_cols].to_dict(orient='records')
X_val = dv.transform(dicts_val)

In [42]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [43]:
y_pred = model.predict_proba(X_val)[:,1]
churn_decision = y_pred >= 0.5
round((y_val == churn_decision).mean(),2)

np.float64(0.7)

In [44]:
y_pred = model.predict(X_val)
y_pred

array([1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1,
       0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 0, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0,
       1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 1, 1, 1,
       1, 1, 0, 1, 0, 1, 0])

In [45]:
round((y_val == y_pred).mean(),2)

np.float64(0.7)

In [46]:
0.74 - (y_val == y_pred).mean()

np.float64(0.04034129692832766)

In [47]:
(y_val == y_pred).mean() - 0.64

np.float64(0.059658703071672314)

In [48]:
original_accuracy = (y_val == y_pred).mean()
original_accuracy

np.float64(0.6996587030716723)

In [49]:
dicts_test = df_test[categorical_cols+numerical_cols].to_dict(orient='records')
X_test = dv.transform(dicts_test)
y_pred = model.predict(X_test)
acc = (y_test == y_pred).mean()
print(acc, round(acc,2))

0.726962457337884 0.73


0.74

## Question 5
- Let's find the least useful feature using the feature elimination technique.
- Train a model using the same features and parameters as in Q4 (without rounding).
- Now exclude each feature from this set and train a model without it. Record the accuracy for each model.
- For each feature, calculate the difference between the original accuracy and the accuracy without the feature.

- Which of following feature has the smallest difference?
    - 'industry'
    - 'employment_status'
    - 'lead_score'

In [50]:
categorical_cols

['lead_source', 'industry', 'employment_status', 'location']

In [51]:
numerical_cols

['number_of_courses_viewed',
 'annual_income',
 'interaction_count',
 'lead_score']

In [52]:
# without 'industry'
dicts_train1 = df_train[['lead_source', 'employment_status', 'location']+numerical_cols].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train1 = dv.fit_transform(dicts_train1)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train1, y_train)

dicts_val1 = df_val[['lead_source', 'employment_status', 'location']+numerical_cols].to_dict(orient='records')
X_val1 = dv.transform(dicts_val1)
y_pred1 = model.predict(X_val1)

acc1 = (y_val == y_pred1).mean()
print(acc1)
print(original_accuracy - acc1)

0.6996587030716723
0.0


In [53]:
# without 'employment_status'
dicts_train2 = df_train[['lead_source', 'industry', 'location']+numerical_cols].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train2 = dv.fit_transform(dicts_train2)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train2, y_train)

dicts_val2 = df_val[['lead_source', 'industry', 'location']+numerical_cols].to_dict(orient='records')
X_val2 = dv.transform(dicts_val2)
y_pred2 = model.predict(X_val2)

acc2 = (y_val == y_pred2).mean()
print(acc2)
print(original_accuracy - acc2)

0.6962457337883959
0.0034129692832763903


In [54]:
# without 'lead_score'
dicts_train3 = df_train[categorical_cols+['number_of_courses_viewed', 'annual_income', 'interaction_count']].to_dict(orient='records')
dv = DictVectorizer(sparse=False)
X_train3 = dv.fit_transform(dicts_train3)
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)
model.fit(X_train3, y_train)

dicts_val3 = df_val[categorical_cols+['number_of_courses_viewed', 'annual_income', 'interaction_count']].to_dict(orient='records')
X_val3 = dv.transform(dicts_val3)
y_pred3 = model.predict(X_val3)

acc3 = (y_val == y_pred3).mean()
print(acc3)
print(original_accuracy - acc3)

0.7064846416382252
-0.0068259385665528916


lead_score

## Question 6
- Now let's train a regularized logistic regression.
- Let's try the following values of the parameter C: [0.01, 0.1, 1, 10, 100].
- Train models using all the features as in Q4.
- Calculate the accuracy on the validation dataset and round it to 3 decimal digits.
- Which of these C leads to the best accuracy on the validation set?

In [55]:
C = [0.01, 0.1, 1, 10, 100]
for c in C:
    dicts_train = df_train[categorical_cols+numerical_cols].to_dict(orient='records')
    dv = DictVectorizer(sparse=False)
    X_train = dv.fit_transform(dicts_train)
    model = LogisticRegression(C=c, random_state=42)
    model.fit(X_train, y_train)

    dicts_val = df_val[categorical_cols+numerical_cols].to_dict(orient='records')
    X_val = dv.transform(dicts_val)
    y_pred = model.predict(X_val)

    acc = (y_val == y_pred).mean()
    print(c, acc)

0.01 0.7542662116040956
0.1 0.8088737201365188
1 0.7918088737201365
10 0.7918088737201365
100 0.7918088737201365


0.1